In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook

In [2]:
train = pd.read_csv('csi_analyze/dataset/train/subs_csi_train.csv',sep=';')
test = pd.read_csv('csi_analyze/dataset/test/subs_csi_test.csv',sep=';')

In [3]:
sbds =  {'SK_ID': np.int32, 'CELL_LAC_ID': np.int32 , 'DATA_VOL_MB': np.float32, 'START_TIME': np.str}
sbvs = {'SK_ID': np.int32, 'CELL_LAC_ID': np.int32 , 'VOICE_DUR_MIN': np.float32, 'START_TIME': np.str}

In [4]:
sbc = {'SK_ID': np.int32, 'CELL_LAC_ID': np.int32, 'MON': np.str, 'SUM_MINUTES':np.float32, 'SUM_DATA_MB':np.float32,'SUM_DATA_MIN': np.float32}

In [5]:
subs_features_train = pd.read_csv('csi_analyze/dataset/train/subs_features_train.csv',sep=';')
subs_bs_consumption_train = pd.read_csv('csi_analyze/dataset/train/subs_bs_consumption_train.csv',sep=';',decimal=',', dtype=sbc)

subs_bs_data_session_train = pd.read_csv('csi_analyze/dataset/train/subs_bs_data_session_train.csv',sep=';',    decimal=',', dtype=sbds)
subs_bs_voice_session_train = pd.read_csv('csi_analyze/dataset/train/subs_bs_voice_session_train.csv',sep=';',  decimal=',', dtype=sbvs)

In [6]:
subs_features_test = pd.read_csv('csi_analyze/dataset/test/subs_features_test.csv',sep=';')
subs_bs_consumption_test = pd.read_csv('csi_analyze/dataset/test/subs_bs_consumption_test.csv',sep=';',decimal=',', dtype=sbc)

subs_bs_data_session_test = pd.read_csv('csi_analyze/dataset/test/subs_bs_data_session_test.csv',sep=';', decimal=',', dtype=sbds)
subs_bs_voice_session_test = pd.read_csv('csi_analyze/dataset/test/subs_bs_voice_session_test.csv',sep=';', decimal=',', dtype=sbvs)

# sf

In [56]:
subs_features_train['SNAP_DATE'] = subs_features_train['SNAP_DATE'].map(lambda x: x.split('.')[1], na_action='ignore')
subs_features_test['SNAP_DATE'] = subs_features_test['SNAP_DATE'].map(lambda x: x.split('.')[1], na_action='ignore')

In [62]:
%%time
sf_train = []

for user in tqdm_notebook(subs_features_train.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    for month in subs_features_train[subs_features_train.SK_ID == user].SNAP_DATE.values:
        se = subs_features_train[subs_features_train.SK_ID == user][subs_features_train.SNAP_DATE.astype('str') == '{}'.format(month)]
        for col in se:
            if col not in ['SK_ID', 'SNAP_DATE']:
                d[col + '_' + month] = se[col].values[0]
    sf_train.append(d)

CPU times: user 12min 54s, sys: 3.06 s, total: 12min 57s
Wall time: 12min 55s


In [63]:
%%time
sf_test = []

for user in tqdm_notebook(subs_features_test.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    for month in subs_features_test[subs_features_test.SK_ID == user].SNAP_DATE.values:
        se = subs_features_test[subs_features_test.SK_ID == user][subs_features_test.SNAP_DATE.astype('str') == '{}'.format(month)]
        for col in se:
            if col not in ['SK_ID', 'SNAP_DATE']:
                d[col + '_' + month] = se[col].values[0]
    sf_test.append(d)

CPU times: user 18min 42s, sys: 4.14 s, total: 18min 46s
Wall time: 18min 43s


In [64]:
sf_train = pd.DataFrame(sf_train)
sf_test = pd.DataFrame(sf_test)

In [67]:
sf_train.to_csv('data_ready_train/sf_train_new.csv', index=False)
sf_test.to_csv('data_ready_train/sf_test_new.csv', index=False)

In [69]:
# sf_test.user_id = sf_test.user_id.map(lambda x: x[0], na_action='ignore')
# sf_train.user_id = sf_train.user_id.map(lambda x: x[0], na_action='ignore')

# sbds

In [70]:
subs_bs_data_session_train['START_TIME'] = subs_bs_data_session_train['START_TIME'].map(lambda x: x.split()[0].split('.')[1], na_action='ignore')
subs_bs_data_session_test['START_TIME'] = subs_bs_data_session_test['START_TIME'].map(lambda x: x.split()[0].split('.')[1], na_action='ignore')

In [79]:
%%time
sbds_train = []

for user in tqdm_notebook(subs_bs_data_session_train.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    
    for month in subs_bs_data_session_train[subs_bs_data_session_train.SK_ID == user].START_TIME.unique():
        
        se = subs_bs_data_session_train[(subs_bs_data_session_train.SK_ID == user) & (subs_bs_data_session_train.START_TIME == month)]
        for col in se.columns:
            
            if col == 'CELL_LAC_ID':    
                d['cell_id_' + month] = list(se['CELL_LAC_ID'].unique())
                
            elif col == 'DATA_VOL_MB':
                d['data_vol_mb_mean_'+ month] = se['DATA_VOL_MB'].mean()
                d['data_vol_mb_sum_'+ month] = se['DATA_VOL_MB'].sum()
                d['data_vol_mb_std_'+ month] = se['DATA_VOL_MB'].std()
                #d['data_vol_mb_log_sum_'+ month] = np.log(se.sum())
                d['data_vol_mb_median_'+ month] = se['DATA_VOL_MB'].median()
                d['data_vol_mb_min_'+ month] = se['DATA_VOL_MB'].min()
                d['data_vol_mb_max_'+ month] = se['DATA_VOL_MB'].max()
                
    sbds_train.append(d)

CPU times: user 3h 41min 44s, sys: 3min 29s, total: 3h 45min 13s
Wall time: 1h 4min 20s


In [80]:
%%time
sbds_test = []

for user in tqdm_notebook(subs_bs_data_session_test.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    
    for month in subs_bs_data_session_test[subs_bs_data_session_test.SK_ID == user].START_TIME.unique():
        
        se = subs_bs_data_session_test[(subs_bs_data_session_test.SK_ID == user) & (subs_bs_data_session_test.START_TIME == month)]
        for col in se.columns:
            
            if col == 'CELL_LAC_ID':    
                d['cell_id_' + month] = list(se['CELL_LAC_ID'].unique())
                
            elif col == 'DATA_VOL_MB':
                d['data_vol_mb_mean_'+ month] = se['DATA_VOL_MB'].mean()
                d['data_vol_mb_sum_'+ month] = se['DATA_VOL_MB'].sum()
                d['data_vol_mb_std_'+ month] = se['DATA_VOL_MB'].std()
                #d['data_vol_mb_log_sum_'+ month] = np.log(se.sum())
                d['data_vol_mb_median_'+ month] = se['DATA_VOL_MB'].median()
                d['data_vol_mb_min_'+ month] = se['DATA_VOL_MB'].min()
                d['data_vol_mb_max_'+ month] = se['DATA_VOL_MB'].max()
                
    sbds_test.append(d)

CPU times: user 4h 35min 48s, sys: 4min 10s, total: 4h 39min 58s
Wall time: 1h 28min 28s


In [81]:
sbds_train = pd.DataFrame(sbds_train)
sbds_test = pd.DataFrame(sbds_test)

In [113]:
sbds_train['count_cell_03'] = sbds_train['cell_id_03'].map(lambda x: len(x), na_action='ignore')
sbds_train['count_cell_04'] = sbds_train['cell_id_04'].map(lambda x: len(x), na_action='ignore')
sbds_train['count_cell_05'] = sbds_train['cell_id_05'].map(lambda x: len(x), na_action='ignore')

In [116]:
sbds_test['count_cell_03'] = sbds_test['cell_id_03'].map(lambda x: len(x), na_action='ignore')
sbds_test['count_cell_04'] = sbds_test['cell_id_04'].map(lambda x: len(x), na_action='ignore')
sbds_test['count_cell_05'] = sbds_test['cell_id_05'].map(lambda x: len(x), na_action='ignore')
sbds_test['count_cell_02'] = sbds_test['cell_id_02'].map(lambda x: len(x), na_action='ignore')

In [118]:
sbds_train.to_csv('data_ready_train/sbds_train_new.csv', index=False)
sbds_test.to_csv('data_ready_train/sbds_test_new.csv', index=False)

# sbvs

In [27]:
subs_bs_voice_session_train['START_TIME'] = subs_bs_voice_session_train['START_TIME'].map(lambda x: x.split()[0].split('.')[1], na_action='ignore')
subs_bs_voice_session_test['START_TIME'] =  subs_bs_voice_session_test['START_TIME'].map(lambda x: x.split()[0].split('.')[1], na_action='ignore')

In [121]:
subs_bs_voice_session_train['START_TIME'] = subs_bs_voice_session_train['START_TIME'].map(lambda x: x.split()[0].split('.')[1], na_action='ignore')
subs_bs_voice_session_test['START_TIME'] = subs_bs_voice_session_test['START_TIME'].map(lambda x: x.split()[0].split('.')[1], na_action='ignore')

In [141]:
%%time
sbvs_train = []

for user in tqdm_notebook(subs_bs_voice_session_train.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    
    for month in subs_bs_voice_session_train[subs_bs_voice_session_train.SK_ID == user].START_TIME.unique():
        
        se = subs_bs_voice_session_train[(subs_bs_voice_session_train.SK_ID == user) & (subs_bs_voice_session_train.START_TIME == month)]
        for col in se.columns:
            
            if col == 'CELL_LAC_ID':
                lc =  list(se['CELL_LAC_ID'].unique())
                d['cell_id_' + month] = lc
                d['counts_cell_id_' + month] = len(lc)
                
            elif col == 'VOICE_DUR_MIN':
                d['VOICE_DUR_MIN_mean_'+ month] = se['VOICE_DUR_MIN'].mean()
                d['VOICE_DUR_MIN_sum_'+ month] = se['VOICE_DUR_MIN'].sum()
                
    sbvs_train.append(d)

CPU times: user 43min 36s, sys: 5.11 s, total: 43min 42s
Wall time: 43min 40s


In [148]:
%%time
sbvs_test = []

for user in tqdm_notebook(subs_bs_voice_session_test.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    
    for month in subs_bs_voice_session_test[subs_bs_voice_session_test.SK_ID == user].START_TIME.unique():
        
        se = subs_bs_voice_session_test[(subs_bs_voice_session_test.SK_ID == user) & (subs_bs_voice_session_test.START_TIME == month)]
        for col in se.columns:
            
            if col == 'CELL_LAC_ID':
                lc =  list(se['CELL_LAC_ID'].unique())
                d['cell_id_' + month] = lc
                d['counts_cell_id_' + month] = len(lc)
                
            elif col == 'VOICE_DUR_MIN':
                d['VOICE_DUR_MIN_mean_'+ month] = se['VOICE_DUR_MIN'].mean()
                d['VOICE_DUR_MIN_sum_'+ month] = se['VOICE_DUR_MIN'].sum()
                
    sbvs_test.append(d)

CPU times: user 59min 55s, sys: 8.95 s, total: 1h 4s
Wall time: 1h 1s


In [152]:
sbvs_test = pd.DataFrame(sbvs_test)
sbvs_train = pd.DataFrame(sbvs_train)

In [157]:
set(sbvs_test) - set(sbvs_train)

{'VOICE_DUR_MIN_mean_02',
 'VOICE_DUR_MIN_sum_02',
 'cell_id_02',
 'counts_cell_id_02'}

In [153]:
sbvs_train.to_csv('data_ready_train/sbvs_train_new.csv', index=False)
sbvs_test.to_csv('data_ready_train/sbvs_test_new.csv', index=False)

# sbс

In [182]:
subs_bs_consumption_train.MON = subs_bs_consumption_train.MON.map(lambda x: str(x).split('.')[1], na_action='ignore')
subs_bs_consumption_test.MON = subs_bs_consumption_test.MON.map(lambda x: str(x).split('.')[1], na_action='ignore')

In [200]:
%%time
sbvs_train = []

for user in tqdm_notebook(subs_bs_consumption_train.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    
    for month in subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user].MON.unique():
        
        se = subs_bs_consumption_train[(subs_bs_consumption_train.SK_ID == user) & (subs_bs_consumption_train.MON == month)]
        for col in se.columns:
            
            if col == 'CELL_LAC_ID':
                lc =  list(se['CELL_LAC_ID'].unique())
                d['cell_id_' + month] = lc
                d['counts_cell_id_' + month] = len(lc)
                
            elif col == 'SUM_MINUTES':
                d['SUM_MINUTES_mean_'+ month] = se['SUM_MINUTES'].mean()
                d['SUM_MINUTES_sum_'+ month] = se['SUM_MINUTES'].sum()
                
            elif col == 'SUM_DATA_MB':
                d['SUM_DATA_MB_mean_'+ month] = se['SUM_DATA_MB'].mean()
                d['SUM_DATA_MB_sum_'+ month] = se['SUM_DATA_MB'].sum()
                
            elif col == 'SUM_DATA_MIN':
                d['SUM_DATA_MIN_mean_'+ month] = se['SUM_DATA_MIN'].mean()
                d['SUM_DATA_MIN_sum_'+ month] = se['SUM_DATA_MIN'].sum()
                
    sbvs_train.append(d)

CPU times: user 30min 3s, sys: 2.15 s, total: 30min 5s
Wall time: 30min 4s


In [201]:
%%time
sbvs_test = []

for user in tqdm_notebook(subs_bs_consumption_test.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    
    for month in subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user].MON.unique():
        
        se = subs_bs_consumption_test[(subs_bs_consumption_test.SK_ID == user) & (subs_bs_consumption_test.MON == month)]
        for col in se.columns:
            
            if col == 'CELL_LAC_ID':
                lc =  list(se['CELL_LAC_ID'].unique())
                d['cell_id_' + month] = lc
                d['counts_cell_id_' + month] = len(lc)
                
            elif col == 'SUM_MINUTES':
                d['SUM_MINUTES_mean_'+ month] = se['SUM_MINUTES'].mean()
                d['SUM_MINUTES_sum_'+ month] = se['SUM_MINUTES'].sum()
                
            elif col == 'SUM_DATA_MB':
                d['SUM_DATA_MB_mean_'+ month] = se['SUM_DATA_MB'].mean()
                d['SUM_DATA_MB_sum_'+ month] = se['SUM_DATA_MB'].sum()
                
            elif col == 'SUM_DATA_MIN':
                d['SUM_DATA_MIN_mean_'+ month] = se['SUM_DATA_MIN'].mean()
                d['SUM_DATA_MIN_sum_'+ month] = se['SUM_DATA_MIN'].sum()
                
    sbvs_test.append(d)

CPU times: user 39min 53s, sys: 2.63 s, total: 39min 55s
Wall time: 39min 54s


In [203]:
sbvs_test = pd.DataFrame(sbvs_test)
sbvs_train = pd.DataFrame(sbvs_train)

In [204]:
sbvs_train.to_csv('data_ready_train/sbc_train_new.csv', index=False)
sbvs_test.to_csv('data_ready_train/sbc_test_new.csv', index=False)